# Introduction
State notebook purpose here

### Get source folder and append to sys directory

In [5]:
from __future__ import print_function
import os
import sys
PROJ_ROOT = os.path.join(os.pardir)
print(os.path.abspath(PROJ_ROOT))
src_dir = os.path.join(PROJ_ROOT, "src")
sys.path.append(src_dir)
# Data path example
#pump_data_path = os.path.join(PROJ_ROOT,
#                              "data",
#                              "raw",
#                              "pumps_train_values.csv")

/mnt/4ba37af6-51fd-47bc-8321-8c500c229114/study/School/KHOA LUAN TOT NGHIEP/runnable_program


### Imports
Import libraries and write settings here.

In [6]:
# Data manipulation
import pandas as pd
import numpy as np
import tensorflow as tf
import sklearn
from tensorflow import keras
from tensorflow.keras import layers
import glob
import xarray as xr
from sklearn.preprocessing import MinMaxScaler
import holidays

# Options for pandas
pd.options.display.max_columns = 50
pd.options.display.max_rows = 30
# Display all cell outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
from IPython import get_ipython
ipython = get_ipython()
# autoreload extension
if 'autoreload' not in ipython.extension_manager.loaded:
    %load_ext autoreload
%autoreload 1
# Use %aimport module to reload each module

# Extract Data
# %aimport features.extract_features
from features import extract_features
%aimport data.create_input_for_models
from data import create_load_transform_processed_data, create_input_for_models
# Make dataset

# Visualizations
import matplotlib.pyplot as plt

# Analysis/Modeling
Do work here

### Get interim data

In [7]:
_interim_data_path = os.path.join(PROJ_ROOT,
                                  "data",
                                  "interim")
_interim_files = glob.glob(_interim_data_path + '/*.csv')

interim_df = pd.DataFrame()
for file in _interim_files:
    print('Currently processing file \n{}'.format(file))
    interim_df = interim_df.append(pd.read_csv(file, parse_dates=True, index_col=['site_id', 'time'],
                                                     dtype={'CO': np.float64, 'NO2': np.float64, 'PM25': np.float64,
                                                            'AQI_h': np.float64, 'AQI_h_I': np.int, 'site_id': np.int}))
# Site 16 have many inconsistency in data so we remove it
interim_df = interim_df[(interim_df.index.get_level_values(0) != 16)]
# Get only columns we need
interim_df = interim_df[['PM25', 'AQI_h', 'AQI_h_Polutant', 'AQI_h_I',
       'AQI_h_label', 'Continous length']]
# Ho Chi Minh data is on site 49
hanoi_df = interim_df[(interim_df.index.get_level_values(0) != 49)].copy()
hcm_df = interim_df[(interim_df.index.get_level_values(0) == 49)].copy()

Currently processing file 
../data/interim/30.csv
Currently processing file 
../data/interim/9.csv
Currently processing file 
../data/interim/32.csv
Currently processing file 
../data/interim/11.csv
Currently processing file 
../data/interim/40.csv
Currently processing file 
../data/interim/28.csv
Currently processing file 
../data/interim/49.csv
Currently processing file 
../data/interim/46.csv
Currently processing file 
../data/interim/10.csv
Currently processing file 
../data/interim/8.csv
Currently processing file 
../data/interim/25.csv
Currently processing file 
../data/interim/7.csv
Currently processing file 
../data/interim/16.csv
Currently processing file 
../data/interim/42.csv
Currently processing file 
../data/interim/44.csv
Currently processing file 
../data/interim/37.csv
Currently processing file 
../data/interim/1.csv
Currently processing file 
../data/interim/13.csv
Currently processing file 
../data/interim/31.csv
Currently processing file 
../data/interim/26.csv
Curr

### Create input for model from interim data and put in to processed

In [8]:
_model_input_data_path = os.path.join(PROJ_ROOT,
                                   "data",
                                   "model_input",
                                   "final_hcm_05",)
target_hour = [5]
timesteps = [5]
train, y_train = create_input_for_models.create(hcm_df, timesteps=timesteps, target_hour=target_hour, test_output=True, dev_output=True, output_path=_model_input_data_path, PROJ_ROOT=PROJ_ROOT)

4
5
0 days 09:00:00


KeyboardInterrupt: 

In [7]:
train[0]

array([[-6.6412216e-01,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
         0.0000000e+00,  0.0000000e+00,  8.8788521e-01,  4.6006504e-01,
         8.6602539e-01,  5.0000000e-01, -2.4492937e-16,  1.0000000e+00,
         9.9853343e-01,  5.4138910e-02,  0.0000000e+00,  0.0000000e+00,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  1.0000000e+00,
         1.0000000e+00, -5.7251906e-01, -5.6488550e-01, -5.7251906e-01,
        -5.7633591e-01, -5.7862598e-01, -5.8396947e-01, -5.9214830e-01,
        -5.6870228e-01, -5.5216283e-01, -5.4122138e-01, -5.3712702e-01,
        -5.3712702e-01, -5.3712702e-01],
       [-6.5648854e-01,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
         0.0000000e+00,  0.0000000e+00,  9.7908407e-01,  2.0345601e-01,
         8.6602539e-01,  5.0000000e-01, -2.4492937e-16,  1.0000000e+00,
         9.9853343e-01,  5.4138910e-02,  0.0000000e+00,  0.0000000e+00,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  1.0000000e+00,
         1.0000000e+00,

In [8]:
y_train

array([-0.648855  , -0.7251908 , -0.77862597, ..., -0.8015267 ,
       -0.8167939 , -0.8015267 ], dtype=float32)

# Results
Show graphs and stats here

# Conclusions and Next Steps
Summarize findings here